In [83]:
%reload_ext autoreload
%autoreload 2

In [84]:
from kret_studies import *
from kret_studies.notebook import *
from kret_studies.complex import *

logger = get_notebook_logger()

In [85]:
from kret_studies.kret_torch.nn_mse_ce import LinearNN


class HousingNN(LinearNN):
    def set_model(self, input_dim: int):
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

In [86]:
model = HousingNN(patience=5000)
# model._wandb = False

In [87]:
run = start_wandb_run(model.name(), project="housing-prices-dataset")

Eval loss,▁
Eval r2,▁
Train loss,▁
Eval loss,1734497271808.0
Eval r2,0.65685
Train loss,980251944640.5872


In [88]:
model.FullStateDict

{'state': {'best_eval_loss': inf,
  'best_eval_r2': -1.0,
  'best_accuracy': -1.0,
  'best_f1': -1.0,
  'epochs_trained': 0},
 'hparams': {'lr': 0.001,
  'gamma': 0.1,
  'stepsize': 7,
  'batchsize': 128,
  'patience': 5000,
  'improvement_tol': 0.0001}}

In [89]:
handle = "yasserh/housing-prices-dataset"
file_path = "Housing.csv"

df: pd.DataFrame = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, handle, file_path)

In [90]:
uks_pd.data_cleanup(df)
df = uks_pd.one_hot_encode(df)
df = uks_pd.move_columns(df, end=["price"])
X, y = uks_pd.split_x_y(df, "price")

/Users/Akseldkw/coding/kretsinger/kret_studies/kret_pandas.py:139: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(non_null, errors="coerce")


In [91]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
model.set_model(input_dim=X.shape[1])
model.post_init()

[2025-11-13 19:25:41,682 | INFO | HousingNN_v000 ] Loaded model weights and state from /Users/Akseldkw/coding/kretsinger/data/pytorch/HousingNN_v000.
[2025-11-13 19:25:41,682 | INFO | HousingNN_v000 ] Full State:
{'hparams': {'batchsize': 128,
             'gamma': 0.1,
             'improvement_tol': 0.0001,
             'lr': 0.001,
             'patience': 5000,
             'stepsize': 7},
 'state': {'best_accuracy': '-100.00%',
           'best_eval_loss': 1675662458880.0,
           'best_eval_r2': '66.85%',
           'best_f1': '-100.00%',
           'epochs_trained': 37362}}


In [93]:
model.model_state

{'best_eval_loss': 1675662458880.0,
 'best_eval_r2': 0.6684711575508118,
 'best_accuracy': -1.0,
 'best_f1': -1.0,
 'epochs_trained': 37362}

In [94]:
model.FullStateDict["state"]

{'best_eval_loss': 1675662458880.0,
 'best_eval_r2': 0.6684711575508118,
 'best_accuracy': -1.0,
 'best_f1': -1.0,
 'epochs_trained': 37362}

In [101]:
model.train_model((X_train, y_train), (X_val, y_val), epochs=1_000)

100%|██████████| 1000/1000 [00:19<00:00, 51.10it/s]


In [102]:
model.FullStateDict

{'state': {'best_eval_loss': 1675662458880.0,
  'best_eval_r2': 0.6684711575508118,
  'best_accuracy': -1.0,
  'best_f1': -1.0,
  'epochs_trained': 39462},
 'hparams': {'lr': 0.001,
  'gamma': 0.1,
  'stepsize': 7,
  'batchsize': 128,
  'patience': 5000,
  'improvement_tol': 0.0001}}

In [103]:
y.dtypes

dtype('int64')

In [104]:
model.FullStateDictDisplay

{'state': {'best_eval_loss': 1675662458880.0,
  'best_eval_r2': '66.85%',
  'best_accuracy': '-100.00%',
  'best_f1': '-100.00%',
  'epochs_trained': 39462},
 'hparams': {'lr': 0.001,
  'gamma': 0.1,
  'stepsize': 7,
  'batchsize': 128,
  'patience': 5000,
  'improvement_tol': 0.0001}}

In [105]:
# raise Exception("stop here")
run.finish()

Eval loss,▃▁█▆▄▂▁▄▃▇▇▅▄▆▇▅█▆▆▇▄▇▄▄▅▁▆▅▅▇▃▅▂▆▇▄▇▃▃▇
Eval r2,▇▁▇▅▄▅▅▄█▆▅▆▅▆▄▅█▃▅▅▅▆▅▅▆▆▆▄▄▅▃▇▅▄▆▅▇▅▃▅
Train loss,▅█▆▄▇▆█▆▆▄▆▆▂▅█▂▄▃▃▁▄▄▃▆▃▂▃▂▂▇▄▄▁▃▄▂▂▂▁▂
Eval loss,1706015064064.0
Eval r2,0.66248
Train loss,973026589818.1284
